In [8]:
import pandas as pd
import numpy as np
import os

In [9]:
input_file_name = 'data-v2.xlsx'
file_path = os.path.join(os.getcwd(), 'input-files', input_file_name)
read_columns = ['Time_stamp', 'SOLAR_GHI_GROUND_01_AVG', 'SOLAR_DIRECTRADIATION_GROUND_01_AVG', 'SOLAR_DIFFUSERADIATION_GROUND_01_AVG', 'DAY_OF_YEAR', 'DECLINATION', 'EOT', 'TC', 'LST', 'HOUR_ANGLE', 'ZENITH_ANGLE']
data_DF = pd.read_excel(file_path, usecols=read_columns)
data_DF.head()

,Time_stamp,SOLAR_DIFFUSERADIATION_GROUND_01_AVG,SOLAR_DIRECTRADIATION_GROUND_01_AVG,SOLAR_GHI_GROUND_01_AVG,DAY_OF_YEAR,DECLINATION,EOT,TC,LST,HOUR_ANGLE,ZENITH_ANGLE
0,2011-10-20 12:20:00,489.455933,415.904572,652.485291,293,-11.403095,15.353572,-0.10904,12.224293,3.364393,39.995685
1,2011-10-20 12:30:00,565.097412,426.994171,578.830566,293,-11.403095,15.353572,-0.10904,12.390960,5.864393,40.264586
2,2011-10-20 12:40:00,479.684174,327.758423,492.760529,293,-11.403095,15.353572,-0.10904,12.557626,8.364393,40.675670
3,2011-10-20 12:50:00,246.236923,306.314087,475.715454,293,-11.403095,15.353572,-0.10904,12.724293,10.864393,41.224658
4,2011-10-20 13:00:00,412.337006,90.560120,482.639404,293,-11.403095,15.353572,-0.10904,12.890960,13.364393,41.906097


In [15]:
# data_DF.head()
# data_DF.dtypes
# data_DF.isnull().sum()
if data_DF.isnull().values.any():
    data_DF = data_DF.dropna()
# data_DF.duplicated().sum()
# data_DF.describe()

In [16]:
# Calculation for I0n
# Extraterrestrial irradiance on a normal surface 
# I0n = Isc(1 + 0.033 * cos( 360 * n/ 365))
# Isc​ is the solar constant (1367 W/m²)
# n is the day of the year (from the date in your period column)

data_DF['I0n'] = 1367 * (1 + 0.033 * np.cos(2 * np.pi * data_DF['DAY_OF_YEAR'] / 365))
data_DF['COS_ZENITH'] = np.cos(np.deg2rad(data_DF['ZENITH_ANGLE']))
data_DF.rename(columns={
    'SOLAR_GHI_GROUND_01_AVG': 'GHI', 
    'SOLAR_DIRECTRADIATION_GROUND_01_AVG': 'DNI', 
    'SOLAR_DIFFUSERADIATION_GROUND_01_AVG': 'DHI'
}, inplace=True)
data_DF['Kt'] = data_DF['GHI'] / (data_DF['I0n'] * data_DF['COS_ZENITH'])
data_DF['Kd'] = data_DF['DHI'] / data_DF['GHI']
data_DF['Kn'] = data_DF['DNI'] / data_DF['I0n']

# # Horizontal extraterrestrial irradiance G0h
data_DF['G0h'] = data_DF['I0n'] * data_DF['COS_ZENITH']
# data_DF.head()
data_DF.describe()

c:\Users\zeesh\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Time_stamp,DHI,DNI,GHI,DAY_OF_YEAR,DECLINATION,EOT,TC,LST,HOUR_ANGLE,ZENITH_ANGLE,I0n,COS_ZENITH,Kt,Kd,Kn,G0h
count,22765,22765.000000,22765.000000,22765.000000,22765.000000,22765.000000,22765.000000,22765.000000,22765.000000,22765.000000,22765.000000,22765.000000,22765.000000,22765.000000,2.276500e+04,22765.000000,22765.000000
mean,2012-03-29 03:02:51.236547584,185.817093,267.553962,369.390392,164.022535,1.713647,-1.403138,-0.388319,11.453800,-8.193003,56.669997,1365.252084,0.504684,0.493344,inf,0.195485,685.757798
min,2011-10-20 12:20:00,0.000000,0.000000,0.000000,1.000000,-23.449783,-14.256968,-0.602549,-0.514623,-187.719344,5.060667,1321.890671,-0.994404,-56.609481,0.000000e+00,0.000000,-1403.882927
25%,2012-01-14 11:10:00,88.415680,20.810774,111.012604,81.000000,-17.381656,-6.544467,-0.474008,8.358913,-54.616307,41.002677,1329.184364,0.277498,0.276009,4.239986e-01,0.015428,380.428515
50%,2012-04-04 05:50:00,182.108795,248.176224,345.450500,153.000000,5.596876,-2.631441,-0.408791,11.465781,-8.013289,56.389901,1364.089959,0.553538,0.535105,6.111101e-01,0.181445,761.906143
75%,2012-06-13 08:00:00,266.242798,461.711731,585.584717,220.000000,19.602452,3.325544,-0.309508,14.570040,38.550599,73.889044,1400.825419,0.754679,0.653901,9.043194e-01,0.337876,1025.720329
max,2012-08-19 18:30:00,645.252136,869.604736,1069.615234,365.000000,23.449783,16.379094,-0.091948,23.446086,171.691291,173.935483,1412.111000,0.996102,167.828181,inf,0.629050,1319.657802
std,NaN,121.831305,230.644918,276.211538,103.302278,17.873005,8.324307,0.138738,3.769976,56.549646,24.505251,34.345269,0.332541,2.285440,NaN,0.168285,448.650534


In [17]:
# K-tests
k_test_condition_1 = (data_DF['GHI'] > 50) & (data_DF['ZENITH_ANGLE'] < 75) & (data_DF['Kd'] < 1.05)
k_test_condition_2 = (data_DF['GHI'] > 50) & (data_DF['ZENITH_ANGLE'] > 75) & (data_DF['Kd'] < 1.10)
k_test_condition_3 = (data_DF['Kn'] < 0.8)
k_test_condition_4 = (data_DF['Kd'] < 0.96) & (data_DF['Kt'] > 0.6)

data_DF['k_test'] = np.where(
    (k_test_condition_1 | k_test_condition_2 ) & k_test_condition_3 & k_test_condition_4,
    'Passed',
    'Failed'
)
data_DF.head()

,Time_stamp,DHI,DNI,GHI,DAY_OF_YEAR,DECLINATION,EOT,TC,LST,HOUR_ANGLE,ZENITH_ANGLE,I0n,COS_ZENITH,Kt,Kd,Kn,G0h,k_test,individual_limit_test,night_time_test
0,2011-10-20 12:20:00,489.455933,415.904572,652.485291,293,-11.403095,15.353572,-0.10904,12.224293,3.364393,39.995685,1381.676507,0.766093,0.616429,0.750141,0.301014,1058.492488,Passed,Passed,Passed
1,2011-10-20 12:30:00,565.097412,426.994171,578.830566,293,-11.403095,15.353572,-0.10904,12.390960,5.864393,40.264586,1381.676507,0.763068,0.549012,0.976274,0.309041,1054.313067,Failed,Passed,Passed
2,2011-10-20 12:40:00,479.684174,327.758423,492.760529,293,-11.403095,15.353572,-0.10904,12.557626,8.364393,40.675670,1381.676507,0.758411,0.470246,0.973463,0.237218,1047.878899,Failed,Passed,Passed
3,2011-10-20 12:50:00,246.236923,306.314087,475.715454,293,-11.403095,15.353572,-0.10904,12.724293,10.864393,41.224658,1381.676507,0.752131,0.457770,0.517614,0.221697,1039.202231,Failed,Passed,Passed
4,2011-10-20 13:00:00,412.337006,90.560120,482.639404,293,-11.403095,15.353572,-0.10904,12.890960,13.364393,41.906097,1381.676507,0.744240,0.469357,0.854338,0.065544,1028.299580,Failed,Passed,Passed


In [18]:
#  Individual Limits Test (ILT)
ILT_condition_1 = (data_DF['GHI'] > -4) & (data_DF['GHI'] < (1.5 * data_DF['I0n'] * data_DF['COS_ZENITH']**1.2 + 100))
ILT_condition_2 = (data_DF['DHI'] > -4) & (data_DF['DHI'] < (0.97 * data_DF['I0n'] * data_DF['COS_ZENITH']**1.2 + 50))
ILT_condition_3 = (data_DF['DNI'] > -4) & (data_DF['DNI'] < data_DF['I0n'])
ILT_condition_4 = (data_DF['DHI'] < 0.8 * data_DF['G0h'])
ILT_condition_5 = (data_DF['GHI'] - data_DF['DHI'] < data_DF['G0h'])

data_DF['individual_limit_test'] = np.where(
    ILT_condition_1 & ILT_condition_2 & ILT_condition_3 & ILT_condition_4 & ILT_condition_5,
    'Passed',
    'Failed'
)
data_DF.head()

,Time_stamp,DHI,DNI,GHI,DAY_OF_YEAR,DECLINATION,EOT,TC,LST,HOUR_ANGLE,ZENITH_ANGLE,I0n,COS_ZENITH,Kt,Kd,Kn,G0h,k_test,individual_limit_test,night_time_test
0,2011-10-20 12:20:00,489.455933,415.904572,652.485291,293,-11.403095,15.353572,-0.10904,12.224293,3.364393,39.995685,1381.676507,0.766093,0.616429,0.750141,0.301014,1058.492488,Passed,Passed,Passed
1,2011-10-20 12:30:00,565.097412,426.994171,578.830566,293,-11.403095,15.353572,-0.10904,12.390960,5.864393,40.264586,1381.676507,0.763068,0.549012,0.976274,0.309041,1054.313067,Failed,Passed,Passed
2,2011-10-20 12:40:00,479.684174,327.758423,492.760529,293,-11.403095,15.353572,-0.10904,12.557626,8.364393,40.675670,1381.676507,0.758411,0.470246,0.973463,0.237218,1047.878899,Failed,Passed,Passed
3,2011-10-20 12:50:00,246.236923,306.314087,475.715454,293,-11.403095,15.353572,-0.10904,12.724293,10.864393,41.224658,1381.676507,0.752131,0.457770,0.517614,0.221697,1039.202231,Failed,Passed,Passed
4,2011-10-20 13:00:00,412.337006,90.560120,482.639404,293,-11.403095,15.353572,-0.10904,12.890960,13.364393,41.906097,1381.676507,0.744240,0.469357,0.854338,0.065544,1028.299580,Failed,Passed,Passed


In [19]:
# Night-Time Zero Testing
filter_condition_1 = data_DF['GHI'] > 5
filter_condition_2 = data_DF['ZENITH_ANGLE'] < 85
data_DF['night_time_test'] = np.where(
    filter_condition_1 & filter_condition_2,
    'Passed',
    'Failed'
)
output_file_name = 'output-v2.csv'
output_path = os.path.join(os.getcwd(), 'output-files', output_file_name)
data_DF.to_csv(output_path, encoding='utf-8', index=None)
data_DF.head()

,Time_stamp,DHI,DNI,GHI,DAY_OF_YEAR,DECLINATION,EOT,TC,LST,HOUR_ANGLE,ZENITH_ANGLE,I0n,COS_ZENITH,Kt,Kd,Kn,G0h,k_test,individual_limit_test,night_time_test
0,2011-10-20 12:20:00,489.455933,415.904572,652.485291,293,-11.403095,15.353572,-0.10904,12.224293,3.364393,39.995685,1381.676507,0.766093,0.616429,0.750141,0.301014,1058.492488,Passed,Passed,Passed
1,2011-10-20 12:30:00,565.097412,426.994171,578.830566,293,-11.403095,15.353572,-0.10904,12.390960,5.864393,40.264586,1381.676507,0.763068,0.549012,0.976274,0.309041,1054.313067,Failed,Passed,Passed
2,2011-10-20 12:40:00,479.684174,327.758423,492.760529,293,-11.403095,15.353572,-0.10904,12.557626,8.364393,40.675670,1381.676507,0.758411,0.470246,0.973463,0.237218,1047.878899,Failed,Passed,Passed
3,2011-10-20 12:50:00,246.236923,306.314087,475.715454,293,-11.403095,15.353572,-0.10904,12.724293,10.864393,41.224658,1381.676507,0.752131,0.457770,0.517614,0.221697,1039.202231,Failed,Passed,Passed
4,2011-10-20 13:00:00,412.337006,90.560120,482.639404,293,-11.403095,15.353572,-0.10904,12.890960,13.364393,41.906097,1381.676507,0.744240,0.469357,0.854338,0.065544,1028.299580,Failed,Passed,Passed
